In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import tensorflow as tf

url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
csv_path = tf.keras.utils.get_file("twitter_sentiment.csv", url)
df = pd.read_csv(csv_path)

df = df[["tweet", "label"]]

In [3]:
# Configuración
vocab_size = 10000
max_length = 100
oov_token = "<OOV>"

# Tokenizador
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(df["tweet"])

# Transformar textos a secuencias
sequences = tokenizer.texts_to_sequences(df["tweet"])
padded = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

# Ejemplo
print("Texto original:", df["tweet"].iloc[0])
print("Secuencia:", sequences[0])
print("Secuencia padded:", padded[0])

Texto original:  @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run
Secuencia: [2, 35, 5, 253, 12, 1, 8, 12, 21, 2754, 98, 6656, 96, 256, 257, 96, 8137, 481]
Secuencia padded: [   2   35    5  253   12    1    8   12   21 2754   98 6656   96  256
  257   96 8137  481    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = padded
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)

Train: (25569, 100) (25569,)
Test: (6393, 100) (6393,)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense


In [8]:
def build_rnn(vocab_size=10000, embedding_dim=64, max_length=100):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        SimpleRNN(64),
        Dense(1, activation='sigmoid')
    ])
    return model

rnn_model = build_rnn()
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.summary()

# Entrenamiento
history_rnn = rnn_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

c:\Users\phipe\OneDrive\Escritorio\taller2\taller2\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.9300 - loss: 0.2601 - val_accuracy: 0.9287 - val_loss: 0.2777
Epoch 2/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9295 - loss: 0.2604 - val_accuracy: 0.9287 - val_loss: 0.2580
Epoch 3/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9334 - loss: 0.2455 - val_accuracy: 0.9287 - val_loss: 0.2572
Epoch 4/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.9294 - loss: 0.2562 - val_accuracy: 0.9287 - val_loss: 0.2584
Epoch 5/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.9318 - loss: 0.2497 - val_accuracy: 0.9287 - val_loss: 0.2571


In [9]:
import pickle

with open("history_rnn.pkl", "wb") as f:
    pickle.dump(history_rnn.history, f)